In [ ]:
#the main focus on this project will be to showcase the ability to data clean and do eda using the pandas package with also some data visualization
#included


In [ ]:
#we will first import pandas package which will be mainly used throughout this project for data cleaning and exploration
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [ ]:
#we import the csv file of the dataset into an appropriately named variables "jobs" for this project
jobs = pd.read_csv("Uncleaned_DS_jobs.csv")

In [ ]:
#To make sure it is in a dataframe format
jobs = pd.DataFrame(jobs)

In [ ]:
#check the columns and their data types we are working with. We can also see that there is no null values. 
#I also notice that the columns need a little more cleaning and change to snake based format
jobs.info()

In [ ]:
#to change the columns to a snake based format we use a lambda function to eventually change them to more appropriate names
jobs.rename(columns= lambda header: header.lower().replace(" ","_"), inplace= True)

In [ ]:
#we should make sure it works
jobs.info()

In [ ]:
#we will now check the first 10 columns using the head column to see what are some immediate shown problems
"""We see some things that may need cleaning straight off the beggining such as deletion of the index column and investigate the competitors.
It may be deleted depending on the number of values with responses """
jobs.head(10)

In [ ]:
#we will delet the index columns since they are a duplicate of the index seen in vs code
jobs.drop(columns="index",inplace=True)

In [ ]:
#check if the index column is not there no more index column
jobs.columns

In [ ]:
#we will now have to investigate the abundance of -1 in the competitors column. What percentage makes up the whole column and how many there are
jobs.loc[jobs["competitors"] == "-1","competitors"].value_counts()


In [ ]:
#check the percentage of the value -1 in the column
round((501/672 * 100),2)

In [ ]:
#there is 501 instances of -1 in the making up 75 percent of the column so it is best to drop the column since it is mainly made up of nulls
jobs.drop(columns="competitors",inplace=True)

In [ ]:
#we check if it worked again
jobs.columns

In [ ]:
#check column by columns to see if anything shoul be cleaned. First 100 rows should be good for a check up 
jobs["job_title"][:100]

In [ ]:
"""I ran into a problem where there are some parenthesis with random words and numbers in some but some have Sr in them or relevant information
so we have to be careful. first we should do is check for all the values in the dataset that have parenthesis to determin which
have important information and whcich to delete"""
jobs.loc[jobs["job_title"].str.contains('\([^()]+\)'), ["job_title","company_name"]]

In [ ]:
"""After further analysis I can conclude that the information in the parenthesis is information that should be in the job_description column
the only exception should be the one containing Sr. which just should delete the parenthesis for the title.
only solution can be use the replace function for the specific case while deleting the rest of the parenthesis and meanwhile converting
 every title to lowercase to make a more consistent format"""

#we should replace (Sr.) to Sr. 
jobs["job_title"]=jobs.loc[:,"job_title"].str.replace("(Sr.)","Sr.")

In [ ]:
#delete the rest of the instances with parenthesis
jobs["job_title"]=jobs.loc[:,"job_title"].str.extract('([^()]+)')

In [ ]:
#to follow consistency we should lower all of the values in a column
jobs["job_title"]=jobs.loc[:,"job_title"].str.lower()

In [ ]:
#check if there is any  instance of parenthesis left
jobs.loc[jobs["job_title"].str.contains('\([^()]+\)'), ["job_title","company_name"]]

In [ ]:
#check the second column to see if there any data cleaning left to do in the second column
"""What I caught was instances of (Glassdoor est.) in all the columns"""
jobs["salary_estimate"][500:600]

In [ ]:
#We delete the instances using the previous method in the job title and check if it worked
jobs["salary_estimate"]=jobs.loc[:,"salary_estimate"].str.extract('([^()]+)')
jobs["salary_estimate"][:100]

In [ ]:
#we check the third column. This is a column that may need to be checked. Nothing seems to be wrong not even signs of the -1 null value
jobs.loc[:,"job_description"]
jobs.loc[jobs.loc[:,"job_description"]=="-1"]

In [ ]:
#check the ratings column for any inconsitency. and also null values(-1) which also is empty 
jobs.loc[:,"rating"][:100]
jobs.loc[jobs.loc[:,"rating"]=="-1"]

In [ ]:
#we check the company name columns and see that the ratings are stuck next to them in the right side with a delimiter of "\n"
jobs.loc[:,"company_name"][:10]

In [ ]:
#we will use extract to delete eveything after "\n". The regex = true being used to indicate that the pattern is a regulary expression
jobs["company_name"]=jobs.loc[:,"company_name"].str.replace(r"\n\d+(\.\d+)?","", regex=True)

In [ ]:
#we check the first 10 columns to see if it works and it does
jobs["company_name"][:10]

In [ ]:
#check the location column now. We can see that there is some inconsitincies with the data . Some values are "remote" and some are just the state and even one occasion the country
jobs.loc[:,"location"][300:400]

In [ ]:
"""A solution that can be done is split the column into 3 parts being state location, city location and possibly even location country while also making a dummy variable 
column or section for remote. For this specific project it is assumed all the locations besides remote are in the united states taking a glance at the results """
jobs.loc[jobs.loc[:,"location"]=="United States","location"].value_counts()


In [ ]:
jobs.loc[jobs.loc[:,"location"]=="Remote","location"].value_counts()

In [ ]:
#lets see if there is any other unique values for state that we mightve missed. We can see that there are some hidden ones with the names of the city instead
#their respective state initials(New_Jersey,Utah,Texas,California) we can use replace to change to their values using a lambda function
jobs.loc[:,"location"].apply(lambda x: x.split(",")[-1]).value_counts()

In [ ]:
#we can use the same lambda function to get the state initials and outliers to a new columns and then replace the values with their repective 
#state initials
jobs["location_state"] = jobs.loc[:,"location"].apply(lambda x: x.split(",")[-1].strip())

In [ ]:
#check if it worked and it did now we do the same with the city and change the indexing to 0 to get the state half
"""Another issue arises where the same states from the previous lines appears the same states which we will switch to n/a values
but first it is better to change the state first"""
jobs["location_city"] = jobs.loc[:,"location"].apply(lambda x: x.split(",")[0].strip())

In [ ]:
#replacing the state names for their respective initials or code names. first we check the values we will replace
jobs["location_state"].value_counts()

In [ ]:
#we will create a functions with the specific state codes then use it in the column
def clean_city(jobs):
    return (
        jobs.loc[:, "location_state"]
        .replace("California","CA")
        .replace("Texas","TX")
        .replace("Utah","UT")
        .replace("New Jersey","NJ")
        .replace("Remote","n/a")
        .replace("United States","n/a")
    )

jobs = jobs.assign(
    location_state = clean_city
)

In [ ]:
#create a new column to see if the location is in the U.S using boolean values. mainly to distinguish between remote and hybrid/on location
jobs["location_in_us"] = jobs.loc[:,"location"].apply(lambda x: 1 if x != "Remote" else 0)


In [ ]:
#we see if it works by printing the results of value_counts before and after the created function and it does works!
jobs["location_state"].value_counts()

In [ ]:
#Now we clean the location city but every non city is now "n/a"
def clean_city(jobs):
    return (
        jobs.loc[:, "location_city"]
        .replace("California","n/a")
        .replace("Texas","n/a")
        .replace("Utah","n/a")
        .replace("New Jersey","n/a")
        .replace("Remote","n/a")
        .replace("United States","n/a")
    )

jobs = jobs.assign(
    location_city = clean_city
)


In [ ]:
#scimming carefully we can see that it worked with 16 "n/a values"
jobs["location_city"].value_counts()

In [ ]:
#check what columns we have left to clean and overall column numbers
jobs.info()

In [ ]:
#next we check the headquarters location unique values. Here we can see that the majority of headquarters locations are the same as the location
jobs["headquarters"].value_counts()

In [ ]:
#here we will use dummy variable labels for location that matches the headquarters location. 1 if they match and 0 if they dont
jobs["same_location"] = jobs.apply(lambda x: 1 if x.location == x.headquarters else 0, axis = 1)

In [ ]:
#check if it worked for safe measure
jobs["same_location"]

In [ ]:
#now we check the company size. Furst thing that we can notice is the repetetive "employees at every column value". when we use unique we can
#see that there is also -1 and unknonw values, we can replace those to use as "n/a" and have them together rather than seperate
jobs["size"].unique()

In [ ]:
#first thing should be to remove all the intances of employees with replace and -1, and unknown in some lines of code
"""instead of creating 3 seperate lines of code we can instead create a function to do everything we want in a single line"""
def size_cleaning(jobs):
    return (
        jobs.loc[:, "size"]
        .replace("employees","")
        .replace("-1","n/a")
        .replace("Unknown","n/a")
    )

jobs = jobs.assign(
    size = size_cleaning
)

In [ ]:
#we check if the function worked and it did 
jobs["size"].unique()

In [ ]:
#check if there is something worng with the next column and there isnt but we don't neccesarily need the founded column so we will drop it 
#explanation: we won't need it due to the abundance of -1 in the column and it wouldn't really add much unless we have a specific question regarding competetors
jobs["founded"][:100]
jobs.drop(columns="founded",inplace=True)

In [ ]:
#checking the type of ownership column
jobs.loc[:,"type_of_ownership"].value_counts()

In [ ]:
"""Here we can see that there is some weird format that we can change such as nonprofit organization to just nonprofit and company-public to
just public company while changin -1 and unknown to n/a and other simplifications for the column in a function"""

def ownership(jobs):
    return (
        jobs.loc[:, "type_of_ownership"]
        .replace("Nonprofit Organization","Nonprofit")
        .replace("-1","n/a")
        .replace("Unknown","n/a")
        .replace("Company - Public","Public")
        .replace("Company - Private","Private")
        .replace("Other Organization","Other")   
    )

jobs = jobs.assign(
    type_of_ownership = ownership
)

In [ ]:
#check if it works and it does
jobs.loc[:,"type_of_ownership"].value_counts()

In [ ]:
#we will now check the industry column unique values
jobs["industry"].value_counts()

In [ ]:
"""From using unique and value counts we can see that there is an a good amount of -1 values making it the most values and that can be solved
quickly using a lambda function"""
jobs["industry"] = jobs["industry"].apply(lambda x: x.replace("-1","n/a"))

In [ ]:
#check value counts to see if it worked 
jobs["industry"].value_counts()

In [ ]:
#check the sector column and their unique values which can be seen to have around 20
jobs["sector"].unique()

In [ ]:
"""checking value counts we can see that there is some -1 values, while the other values seem to be looking fine. for this we can use 
the lambda function from the previous """
jobs["sector"].value_counts()
jobs["sector"] = jobs["sector"].apply(lambda x: x.replace("-1","n/a"))

In [ ]:
#check if it worked
jobs["sector"].value_counts()

In [ ]:
#we check the revenue
jobs["revenue"].value_counts()

In [ ]:
"""Here we can combine the unknown value and -1 to n/a values in a function. we overall leabe it alone to not create uneccessary columns and again
won't be needed to modyify unless we will need it for a specific problem"""
def revenue_cleanup(jobs):
    return (
        jobs.loc[:, "revenue"]
        .replace("-1","n/a")
        .replace("Unknown / Non-Applicable","n/a")
    )

jobs = jobs.assign(
    revenue = revenue_cleanup
)

In [ ]:
#we check that we have (usd) in the columns so we must use the extract function with the lambda function to get rid of all parenthesis
#then we check if it worked
jobs["revenue"].value_counts()
jobs["revenue"]=jobs.loc[:,"revenue"].str.extract('([^()]+)')
jobs["revenue"].value_counts()

In [ ]:
#Now we use jobs to check if there still any instance of -1 in the whole dataframe. we can see that the headquarters we forgot to delete the 
#instances of -1 in the data frame
jobs[jobs.eq("-1").any(axis = 1)]
jobs.loc[:,"headquarters"][154]

In [ ]:
#there is multiple -1 values in the headquarters column which we can get rid of with the lambda function
jobs.loc[jobs["headquarters"] == "-1","headquarters"].value_counts()

#we will use the lambda function to change -1 to n/a in the headquarters column
jobs["headquarters"] = jobs["headquarters"].apply(lambda x: x.replace("-1","n/a"))

#we will see if it worked 
jobs["headquarters"].value_counts()

In [ ]:
#we will go check if there is -1 value left and there isn't
jobs[jobs.eq("-1").any(axis = 1)]

In [ ]:
"""For the salary estimate we can split them both into 3 new columns using the lowest salary, highest and average salary
getting rid of the k and $ in the parts to do further analysis and for data visualization. we should check the overall structure by checking the
first 10 columns"""
jobs["salary_estimate"][:10]

In [ ]:
#using a custom function we can use to get rid of the "k" and "$" and extract the partswhile also turning the column to numeric all in a couple 
#of lines of code for the lower salary
jobs["lower_salary"] = jobs.loc[:,"salary_estimate"].apply(lambda x: x.split("-")[0].strip())

def lower_salary_clean(jobs):
    return pd.to_numeric(
        jobs.loc[:, "lower_salary"]
        .str.replace("$","")
        .str.replace("K","")
    )
jobs = jobs.assign(
    lower_salary = lower_salary_clean
)

In [ ]:
#check if all is good using the unique value and it does 
jobs["lower_salary"].unique()

In [ ]:
#now we do the same exact for the highest potential salary
jobs["highest_salary"] = jobs.loc[:,"salary_estimate"].apply(lambda x: x.split("-")[-1].strip())

def upper_salary_clean(jobs):
    return pd.to_numeric(
        jobs.loc[:, "highest_salary"]
        .str.replace("$","")
        .str.replace("K","")
    )
jobs = jobs.assign(
    highest_salary = upper_salary_clean
)

In [ ]:
#check if it worked and it did
jobs["highest_salary"].unique()

In [ ]:
#we use the highest and lowest possible salary to get the average salary of the job 
jobs["average_salary"] = (jobs["highest_salary"] + jobs["lower_salary"]) //2

In [ ]:
#check if it worked
jobs["average_salary"].unique()

In [ ]:
#we can see a 43 so we check if everyhing is good and normal which it is
jobs.loc[jobs.loc[:,"average_salary"]== 43,"average_salary"]

In [ ]:
""""Now we will make a column called job_simp for simplification of the job meaning if they had specific words they would be simplified
to a specific title"""
#first we must check what are the most prevelant in the given job titles and from there check which are the data related jobs we want
jobs["job_title"].value_counts()

def title_simplification(title):
    if "data scientist" in title.lower():
        return "data scientist"
    elif "data analyst" in title.lower():
        return "data analyst"
    elif "data engineer" in title.lower():
        return "data engineer"
    elif "machine learning" in title.lower():
        return "machine learning"
    elif "analyst" in title.lower():
        return "data analyst"
    else:
        return "n/a"

#we then apply the function to the column and checj the unique values  
jobs["job_simp"]= jobs["job_title"].apply(title_simplification)
jobs["job_simp"].value_counts()

In [ ]:
#when skimming earlier I saw a good amount of senior, sr. titles in the job titles which can be for more senior position
#we can use the previous function method to distinguish titles in the job using key words
def title_position(title):
    if "jr." in title.lower():
        return "jr"
    elif "sr" in title.lower():
        return "sr"
    elif "sr." in title.lower():
        return "sr"
    elif "vp" in title.lower():
        return "sr"
    else:
        return "n/a"
    

jobs["job_position"]= jobs["job_title"].apply(title_position)   

jobs["job_position"].value_counts()

In [ ]:
#all we have left to do is for search of the most popular skills in the description bar such as python, excel,tableau,power bi
#we can do this by storing them into dummy variable format where 1 is if the word if founded in the description
jobs["python"] = jobs.apply(lambda x: 1 if "python" in x.job_description.lower() else 0, axis= 1)
jobs["excel"] = jobs.apply(lambda x: 1 if "excel" in x.job_description.lower() else 0, axis= 1)
jobs["tableau"] = jobs.apply(lambda x: 1 if "tableau" in x.job_description.lower() else 0, axis= 1)
jobs["power_bi"] = jobs.apply(lambda x: 1 if "power bi" in x.job_description.lower() else 0, axis= 1)
jobs["sql"] = jobs.apply(lambda x: 1 if "sql" in x.job_description.lower() else 0, axis= 1)

jobs["python"].value_counts()
jobs["excel"].value_counts()
jobs["tableau"].value_counts()
jobs["power_bi"].value_counts()
jobs["sql"].value_counts()

In [ ]:
#we will import matplotlib  to beggin EDA and data visualization proccess
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#we create a function where we can get the exact number of the x and y values in a bar graph
def valuelabel(x,y):
    for i in range(len(x)):
        plt.text(i,y[i],y[i], ha = 'center',
                  bbox = dict(facecolor = 'yellow', alpha =0.8))

In [ ]:
#we will first see visually what is the number of the jobs that require python vs the ones that dont in a stacked bar graph
python_1 = len(jobs.loc[jobs["python"]==1,"python"])
sql_1 = len(jobs.loc[jobs["sql"]==1,"sql"])
excel_1 = len(jobs.loc[jobs["excel"]==1,"excel"])
tableau_1 = len(jobs.loc[jobs["tableau"]==1,"tableau"])
power_bi_1 = len(jobs.loc[jobs["power_bi"]==1,"power_bi"])

python_0 = len(jobs.loc[jobs["python"]==0,"python"])
sql_0 = len(jobs.loc[jobs["sql"]==0,"sql"])
excel_0 = len(jobs.loc[jobs["excel"]==0,"excel"])
tableau_0 = len(jobs.loc[jobs["tableau"]==0,"tableau"])
power_bi_0 = len(jobs.loc[jobs["power_bi"]==0,"power_bi"])


x = ["python","excel","sql","tableau","power_bi"]
y1 = [python_1,excel_1,sql_1,tableau_1,power_bi_1]
y2 = [python_0,excel_0,sql_0,tableau_0,power_bi_0]
plt.bar(x,y1, color = 'b')
plt.bar(x, y2, bottom=y1, color='r')
valuelabel(x,y1)
valuelabel(x,y2)
plt.show()

In [ ]:
#we could repeat this into all of the other skillset appearance but it would be a lot of redundacny and a lot of code copy and pasted
#instead we can create a function to where we search the information wanted already graphed for looking the average salary for 
#specific jobs with specific skills

def salary_info():
    command = input("select job option or help for options:").lower()
    if command == "help":
        return print("""job options:
        data scientist
        data analyst
        data engineer
        machine learning
        for non_data_jobs type 'n/a'
      
        technical skill options:
        python
        excel
        sql
        tableau
        """)
    elif command == "exit":
        return 0
    elif command == "data scientist":
        try:
            technical_skill = input("enter interest skill:").lower()
            skill = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==1)]["average_salary"].mean()
            skill =round(skill,2)
            non_required = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==0)]["average_salary"].mean()
            non_required =round(non_required,2)
            y= ([skill,non_required])
            x = (["skill needed","not required"]) 
            plt.bar(x,y)
            plt.ylim(80, 150)
            valuelabel(x,y)
            plt.show()   
        except KeyError:
            print("invalid option, try again!")
    elif command == "data analyst":
        try:
            technical_skill = input("enter interest skill:").lower()
            skill = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==1)]["average_salary"].mean()
            skill =round(skill,2)
            non_required = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==0)]["average_salary"].mean()
            non_required =round(non_required,2)
            y= ([skill,non_required])
            x = (["skill needed","not required"]) 
            plt.bar(x,y)
            plt.ylim(80, 150)
            valuelabel(x,y)
            plt.show()   
        except KeyError:
            print("invalid option, try again!")
    elif command == "data engineer":
        try:
            technical_skill = input("enter interest skill:").lower()
            skill = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==1)]["average_salary"].mean()
            skill =round(skill,2)
            non_required = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==0)]["average_salary"].mean()
            non_required =round(non_required,2)
            y= ([skill,non_required])
            x = (["skill needed","not required"]) 
            plt.bar(x,y)
            plt.ylim(80, 150)
            valuelabel(x,y)
            plt.show()   
        except KeyError:
            print("invalid option, try again!")
    elif command == "machine learning":
        try:
            technical_skill = input("enter interest skill:").lower()
            skill = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==1)]["average_salary"].mean()
            skill =round(skill,2)
            non_required = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==0)]["average_salary"].mean()
            non_required =round(non_required,2)
            y= ([skill,non_required])
            x = (["skill needed","not required"]) 
            plt.bar(x,y)
            plt.ylim(80, 150)
            valuelabel(x,y)
            plt.show()   
        except KeyError:
            print("invalid option, try again!")
    elif command == "n/a":
        try:
            technical_skill = input("enter interest skill:").lower()
            skill = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==1)]["average_salary"].mean()
            skill =round(skill,2)
            non_required = jobs.loc[(jobs["job_simp"]==command)&(jobs[technical_skill]==0)]["average_salary"].mean()
            non_required =round(non_required,2)
            y= ([skill,non_required])
            x = (["skill needed","not required"]) 
            plt.bar(x,y)
            plt.ylim(80, 150)
            valuelabel(x,y)
            plt.show()   
        except KeyError:
            print("invalid option, try again!")
    else:
        print("option not available check menu for options")

salary_info()

In [ ]:
#we will now check which type of job pays the most
ds_mean = round(jobs.loc[jobs["job_simp"]=="data scientist"]["average_salary"].mean(),2)
da_mean = round(jobs.loc[jobs["job_simp"]=="data analyst"]["average_salary"].mean(),2)
de_mean = round(jobs.loc[jobs["job_simp"]=="data engineer"]["average_salary"].mean(),2)
ml_mean = round(jobs.loc[jobs["job_simp"]=="machine learning"]["average_salary"].mean(),2)

label = ["data scientist","data analyst","data engineer","machine learning"]
y = [ds_mean,da_mean,de_mean,ml_mean]
plt.bar(label,y)
plt.ylim(110, 130)
plt.show()